# Procesamiento de Datos a Gran Escala

<p><strong>Objetivo: </strong> El objetivo de este cuaderno es aprender sentencias pyspark para el preprocesamiento de los datos:</p>

## Identificación y tratamiento de valores faltantes

In [1]:
import findspark
findspark.init()
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext

In [ ]:
# Configuración local
configuraDierickBrochero = (
    SparkConf()
        .set("spark.scheduler.mode", "FAIR")
        .set("spark.executor.memory", "10G")
        .set("spark.executor.cores", "1")
        .set("spark.cores.max", "9")
        .set("spark.ui.port", "4040")
        .setMaster("local[*]")  # <---- Spark local usando todos los cores
        .setAppName("hpcsparkDierickBrochero")
)

# Crear sesión Spark local
sparkDierickBrochero = (
    SparkSession
        .builder
        .config(conf=configuraDierickBrochero)
        .getOrCreate()
)

sqlContext = SQLContext(sparkDierickBrochero.sparkContext)

sparkDierickBrochero

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/26 23:35:23 WARN Utils: Your hostname, MacBook-Pro-de-Dierick.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
25/11/26 23:35:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/26 23:35:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/homebrew/opt/apache-spark/libexec/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52757)
Traceback (most recent call last):
  File "/Users/dierickbr/.pyenv/versions/3.13.3/lib/python3.13/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dierickbr/.pyenv/versions/3.13.3/lib/python3.13/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dierickbr/.pyenv/versions/3.13.3/lib/python3.13/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
    ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dierickbr/.pyenv/versions/3.13.3/lib/python3.13/socketserver.py", line 766, in __init__
    self.handle()
    ~~~~~~~~~~~^^
  File "/opt/homebrew/opt/apache-spark/libexec/python/py

In [3]:
df = sparkDierickBrochero.createDataFrame(
[
('Store 1',1,448),
('Store 1',2,None),
('Store 1',3,499),
('Store 1',44,432),
(None,None,None),
('Store 2',1,355),
('Store 2',1,355),
('Store 2',None,345),
('Store 2',3,387),
('Store 2',4,312),
],
['Store','WeekInMonth','Revenue']
)

Indentificación

In [4]:
display(df.filter(df.Revenue.isNull()).show())

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          2|   NULL|
|   NULL|       NULL|   NULL|
+-------+-----------+-------+



None

In [5]:
from pyspark.sql.functions import count, when, isnull
display(df.select(
[count(when(isnull(c), c)).alias(c) for c in df.columns]
))

DataFrame[Store: bigint, WeekInMonth: bigint, Revenue: bigint]

Eliminado registros con valores faltantes

In [6]:
df2 = df.dropna()
display(df2.show())

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          3|    499|
|Store 1|         44|    432|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



None

In [7]:
df2 = df.dropna('all')
display(df2.show())

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|   NULL|
|Store 1|          3|    499|
|Store 1|         44|    432|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|       NULL|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



None

There is one important thing to note about fillna – it’ll only do the exchange
operation for matching column types. So if you use a numeric value for a string column
or the other way around, it won’t work.

Sustituyendo por un valor:

In [8]:
display(df.fillna(0).show())
display(df.fillna(0, ['Revenue']).show())
display(df.fillna({'WeekInMonth' : 2, 'Revenue' : 3}).show())

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|      0|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   NULL|          0|      0|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|          0|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



None

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|      0|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   NULL|       NULL|      0|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|       NULL|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



None

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|      3|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   NULL|          2|      3|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|          2|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



None

Sustituyendo con la media

In [9]:
from pyspark.sql.functions import mean
df.select(mean(df.Revenue)).show()

+------------+
|avg(Revenue)|
+------------+
|     391.625|
+------------+



In [10]:
display(df.fillna(391.625, ['Revenue']))

DataFrame[Store: string, WeekInMonth: bigint, Revenue: bigint]

## Eliminando duplicados

In [11]:
display(df.dropDuplicates())

DataFrame[Store: string, WeekInMonth: bigint, Revenue: bigint]

In [12]:
display(df.dropDuplicates(['Store','WeekInMonth']))

DataFrame[Store: string, WeekInMonth: bigint, Revenue: bigint]

## Eliminando columnas

In [13]:
display(df.drop('Revenue'))
display(df.drop('Revenue','Store'))

DataFrame[Store: string, WeekInMonth: bigint]

DataFrame[WeekInMonth: bigint]

## Identificando y resolviendo valores inconsistentes

In [14]:
display(df.show())

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|   NULL|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   NULL|       NULL|   NULL|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|       NULL|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



None

In [15]:
display(df.filter(df.Store == 'Store 1').describe().show())

+-------+-------+-----------------+-----------------+
|summary|  Store|      WeekInMonth|          Revenue|
+-------+-------+-----------------+-----------------+
|  count|      4|                4|                3|
|   mean|   NULL|             12.5|459.6666666666667|
| stddev|   NULL|21.01586702153082|34.99047489436709|
|    min|Store 1|                1|              432|
|    max|Store 1|               44|              499|
+-------+-------+-----------------+-----------------+



None

Esto dará el valor en un cuantil dado, en el intervalo de 0 a 1. Por lo tanto, si establece el segundo argumento en 0.0, obtendrá el valor más bajo para la columna. Con 1.0 obtienes el valor más alto. En el medio tienes la mediana, que es lo que se está buscando:

In [16]:
print(df.approxQuantile('Revenue', [0.5], 0))

[355.0]


## Pivot

A veces, desea cambiar sus datos de filas a columnas. La función se llama pivotar y está disponible en Pyspark.

Básicamente, estás rotando los datos alrededor de un eje determinado, de ahí el nombre.

En este caso, ese eje son los datos en una de sus columnas.

In [17]:
df_pivoted = df.groupBy('WeekInMonth').pivot('Store').sum('Revenue').orderBy('WeekInMonth')
display(df_pivoted)

DataFrame[WeekInMonth: bigint, null: bigint, Store 1: bigint, Store 2: bigint]

In [18]:
display(df
.groupBy('Store','WeekInMonth')
.sum('Revenue')
.orderBy('WeekInMonth'))

DataFrame[Store: string, WeekInMonth: bigint, sum(Revenue): bigint]

In [19]:
display(df_pivoted.withColumnRenamed('Store 1','Store1')
        .withColumnRenamed('Store 2','Store2')
        .selectExpr('WeekInMonth',"stack(2, 'Store 1', Store1, 'Store 2', Store2) as (Store,Revenue)"))

DataFrame[WeekInMonth: bigint, Store: string, Revenue: bigint]

## Explode

Hay otra situación con la que te encontrarás de vez en cuando. A veces llegan varios puntos de datos juntos en una columna. Esto usual cuando JSON es el formato de origen.

Puede resolver este problema utilizando el comando de Explode. Tomará la cadena con varios valores y los colocará en una fila cada uno.

In [20]:
from pyspark.sql.functions import explode
df = sparkDierickBrochero.createDataFrame([
(1, ['Rolex','Patek','Jaeger']),
(2, ['Omega','Heuer']),
(3, ['Swatch','Rolex'])],
('id','watches'))
display(df.withColumn('watches',explode(df.watches)).show())

+---+-------+
| id|watches|
+---+-------+
|  1|  Rolex|
|  1|  Patek|
|  1| Jaeger|
|  2|  Omega|
|  2|  Heuer|
|  3| Swatch|
|  3|  Rolex|
+---+-------+



None

## Normalización

In [21]:
%fs ls /databricks-datasets/definitive-guide/data/simple-ml-scaling

UsageError: Line magic function `%fs` not found.


In [32]:
scaleDF = sparkDierickBrochero.read.parquet("/databricks-datasets/definitive-guide/data/simple-ml-scaling")
display(scaleDF)

25/11/19 22:33:34 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: /databricks-datasets/definitive-guide/data/simple-ml-scaling.
java.io.FileNotFoundException: File /databricks-datasets/definitive-guide/data/simple-ml-scaling does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(Res

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/databricks-datasets/definitive-guide/data/simple-ml-scaling. SQLSTATE: 42K03

In [ ]:
from pyspark.ml.feature import MinMaxScaler
minMax = MinMaxScaler().setMin(5).setMax(10).setInputCol("features")
fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show()

In [ ]:
from pyspark.ml.feature import StandardScaler
sScaler = StandardScaler().setInputCol("features")
sScaler.fit(scaleDF).transform(scaleDF).show()